In [7]:
#Entête/importation des données 
import flask

import sqlite3 as sl 

connexion = sl.connect("site_3D_1.db")

curseur = connexion.cursor()


In [10]:
curseur.execute ("PRAGMA foreign_keys = ON")
curseur.execute ("DROP TABLE IF EXISTS Fichier_stl")
curseur.execute ("DROP TABLE IF EXISTS Client")
curseur.execute ("DROP TABLE IF EXISTS tags")
curseur.execute ("DROP TABLE IF EXISTS fichier_tags")


In [12]:
#Question 1+2 : ----------

curseur.execute ('''CREATE TABLE Fichier_stl(
code_variable CHAR (8) PRIMARY KEY,
libellé VARCHAR(30),
val_defaut INTEGER,
val_min INTEGER,
val_max INTEGER,
description VARCHAR (190),
chemin_serv TEXT
)
''')


curseur.execute ('''CREATE TABLE Client(
identifiantC VARCHAR (15) PRIMARY KEY,
nom VARCHAR (90), 
adresse VARCHAR (110),
date_commande VARCHAR (40),
date_livraison VARCHAR (40),
paramètres_choisis VARCHAR (200),
FOREIGN KEY (paramètres_choisis) REFERENCES Fichier_stl(code_variable)
)
''')

curseur.execute('''CREATE TABLE tags (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nom TEXT UNIQUE NOT NULL
)
''')

curseur.execute('''CREATE TABLE fichier_tags (
    fichier_id INTEGER,
    tag_id INTEGER,
    FOREIGN KEY (fichier_id) REFERENCES fichiers_stl(code_variable),
    FOREIGN KEY (tag_id) REFERENCES tags(id)
)
''')



In [14]:
donnees1 = [("C1234567", "Coeur3D", 8, 6, 15, "coeur en 3D", "modèles/12.scad")]

curseur.executemany("INSERT INTO Fichier_stl (code_variable, libellé, val_defaut, val_min, val_max, description, chemin_serv) VALUES ( ?, ?, ?, ?, ?, ?, ?) ",
donnees1)

In [16]:
#Question 1+2 : ----------

donnees = [("LDV123", "Léonard de Vinci", "Rome", "16/02/2025", "24/02/2025", "C1234567")]

curseur.executemany("INSERT INTO Client (identifiantC, nom, adresse, date_commande, date_livraison, paramètres_choisis) VALUES ( ?, ?, ?, ?, ?, ?) ",
donnees)

In [18]:
#Fonction permettant de récupérer les informations d’un fichier donnée en utilisant son identifiant.
def info_fichier(code) :
    curseur.execute("SELECT * FROM Fichier_stl WHERE code_variable = ?", (code,))
    print ("Informations fichier-------\n", curseur.fetchall())

In [20]:
info_fichier("C1234567")

Informations fichier-------
 [('C1234567', 'Coeur3D', 8, 6, 15, 'coeur en 3D', 'modèles/12.scad')]


In [41]:
#Question 5 :Proposez une fonction permettant d'obtenir la liste des oeuvres d’art dans une exposition donnée. 
#Affichez les descriptions des oeuvres d’art et le lieu de l’exposition.
def info_expo(c_client) :
    curseur.execute("SELECT F.val_defaut, F.val_min, F.val_max FROM Fichier_stl as F INNER JOIN Client as C ON C.paramètres_choisis = F.code_variable WHERE identifiantC = ?", (c_client,))
    print ("Paramètres choisis par le client-------\n", curseur.fetchall())

In [43]:
info_expo("LDV123")

Paramètres choisis par le client-------
 []


In [45]:
#ENTRAINEMENT,NE PAS TENIR COMPTE SVP.
#Proposez une fonction permettant d'obtenir le total des ventes des oeuvres d’art dans une exposition donnée
#def vente(galerie):
#    curseur.execute("SELECT SUM(prix) FROM vente WHERE vendue= 1 AND galerie_art = ?", (galerie,))
#    print ("Total_vendu-------\n", curseur.fetchall())

In [ ]:
##ENTRAINEMENT,NE PAS TENIR COMPTE SVP.
#Proposez une fonction permettant de récupérer toutes les oeuvres d’art qui sont actuellement en exposition. 
#Affichez les noms des réalisateurs et la durée de l’exposition pour chacune.
#def vente2():
#    curseur.execute("SELECT realisateur, durée FROM vente AS V INNER JOIN Oeuvre_Art AS A ON oeuvre = identifiant INNER JOIN exposition AS E ON identifiantE=identifiantEV WHERE vendue= 0")
#    print ("Oeuvres en expositions-------\n", curseur.fetchall())

In [ ]:
#vente2()

In [ ]:
#ENTRAINEMENT,NE PAS TENIR COMPTE SVP.
#Proposez une fonction permettant de mettre à jour l’état d’une oeuvre d’art à "Vendue" une fois la vente est réalisée.
#def majvendue(identOV: str):
#    curseur.execute("UPDATE vente SET vendue = 1 WHERE identifiantO = ? ",(identOV,))

In [ ]:
#majvendue("IncoMILO")

In [ ]:
#ENTRAINEMENT,NE PAS TENIR COMPTE SVP.
#Proposez une fonction qui permet de rechercher des expositions en fonction de certains critères tels que le lieu, 
#le nom du réalisateur de l’oeuvre d’art, la date de début et de fin de l’exposition. La fonction devrait renvoyer une liste
#des expositions correspondant aux critères de recherche.
#def recherche (lieu, réalisateur, début, fin):
#curseur.execute("SELECT nom_Expo FROM exposition AS E INNER JOIN Oeuvre_Art AS A ON identifiant = identifiantO INNER JOIN vente AS V ON identifiant=identifiantO WHERE lieu= ? AND realisateur = ? AND date_debut = ? AND date_fin= ?", (lieu, réalisateur, début, fin,))
#    v=list(curseur.fetchall())
#    print ("Oeuvres en expositions-------\n", v)


In [ ]:
#recherche ("Paris", "Léonard de Vinci", 1550, 2023)

In [ ]:
#ENTRAINEMENT,NE PAS TENIR COMPTE SVP.
#Question 10 : Créez des fonctions qui génèrent des rapports statistiques sur la gestion des ventes de la galerie d’art, 
#tels que le nombre total de ventes effectuées par exposition, les oeuvre d’art les plus chères, etc.

#def stat(galerie):
#    curseur.execute("SELECT galerie_art, SUM(prix), MAX(prix) FROM vente WHERE galerie_art = ? AND vendue = 1 GROUP BY galerie_art", (galerie,))
#    print("Rapport des ventes de la galerie----------\n", curseur.fetchall())

In [ ]:
#stat("art von Berlin")

In [ ]:
curseur.close()
connexion.close()

In [ ]:
"Par Ismaël DIALLO."

'Par Ismaël DIALLO.'

In [3]:
from flask import Flask, render_template, request, redirect, url_for
from flask_sqlalchemy import SQLAlchemy
from threading import Thread

app = Flask(__name__)

# Database configuration
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)

# Define a model
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)

# Create the database tables
with app.app_context():
    db.create_all()

# Home route to display users and a form
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        name = request.form['name']
        if name:
            user = User(name=name)
            db.session.add(user)
            db.session.commit()
            return redirect(url_for('index'))
    
    users = User.query.all()
    return render_template('index.html', users=users)

# Run Flask in a separate thread
def run_flask():
    app.run(host='127.0.0.1', port=5000, debug=False, use_reloader=False)

flask_thread = Thread(target=run_flask)
flask_thread.start()


In [5]:
html_template = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>User List</title>
    <style>
        body { font-family: Arial, sans-serif; text-align: center; margin-top: 50px; }
        form { margin-bottom: 20px; }
        input { padding: 5px; }
        button { padding: 5px 10px; }
        ul { list-style: none; padding: 0; }
        li { padding: 5px; border-bottom: 1px solid #ddd; }
    </style>
</head>
<body>
    <h1>User List</h1>
    <form method="POST">
        <input type="text" name="name" placeholder="Enter name" required>
        <button type="submit">Add User</button>
    </form>
    <ul>
        {% for user in users %}
            <li>{{ user.name }}</li>
        {% endfor %}
    </ul>
</body>
</html>
"""

# Save the template in the "templates" folder
import os

os.makedirs("templates", exist_ok=True)
with open("templates/index.html", "w") as file:
    file.write(html_template)
